In [5]:
import os
import pandas as pd
import json
from IPython.display import HTML

class DataParser:
    def parse_file(self, filepath):
        _, file_extension = os.path.splitext(filepath)
        data = pd.DataFrame()  # Initialize an empty DataFrame for fallback.
        metadata = {'Error': 'File could not be processed or empty.'}
        try:
            if file_extension.lower() in ['.csv']:
                return self._process_dataframe(pd.read_csv(filepath))
            elif file_extension.lower() in ['.xls', '.xlsx']:
                return self._process_dataframe(pd.read_excel(filepath))
            elif file_extension.lower() in ['.json']:
                return self._process_dataframe(self.parse_json(filepath))
            elif file_extension.lower() in ['.ndjson']:
                return self._process_dataframe(self.parse_ndjson(filepath))
            else:
                raise ValueError(f"Unsupported file type: {file_extension}")
        except Exception as e:
            return pd.DataFrame(), {'Error': f"Error parsing {filepath}: {str(e)}"}

    def parse_json(self, filepath):
        with open(filepath, 'r') as file:
            data = json.load(file)
        try:
            if isinstance(data, list):
                if all(isinstance(item, dict) for item in data):
                    return pd.json_normalize(data)
                else:
                    raise ValueError("JSON list elements are not all dictionaries.")
            elif isinstance(data, dict):
                return pd.json_normalize(data, max_level=1)  # Adjust max_level if needed
            else:
                raise ValueError("JSON structure is not recognized.")
        except Exception as e:
            raise ValueError(f"Error normalizing JSON data: {str(e)}")

    def parse_ndjson(self, filepath):
        data = []
        with open(filepath, 'r') as file:
            for line in file:
                data.append(json.loads(line))
        return pd.json_normalize(data)

    def _process_dataframe(self, df):
        metadata = []
        try:
            if not df.empty:
                for col in df.columns:
                    dtype = df[col].dtype
                    sql_type = self._map_dtype_to_sql(dtype)
                    # Check if column contains lists and handle accordingly
                    if any(isinstance(x, list) for x in df[col].dropna()):
                        max_length = df[col].dropna().apply(lambda x: max([len(str(item)) for item in x] if x else [0])).max()
                    else:
                        max_length = df[col].apply(lambda x: len(str(x)) if pd.notnull(x) else 0).max()
                    metadata.append({'Field Name': col, 'Data Type': sql_type, 'Field Length': max_length})
            else:
                metadata.append({'Field Name': 'Error', 'Data Type': 'N/A', 'Field Length': 'N/A'})
        except Exception as e:
            metadata = [{'Field Name': 'Error', 'Data Type': 'N/A', 'Field Length': f"Processing Error: {str(e)}"}]
        return df.head(10), metadata

    def _map_dtype_to_sql(self, dtype):
        if pd.api.types.is_integer_dtype(dtype):
            return 'INT'
        elif pd.api.types.is_float_dtype(dtype):
            return 'FLOAT'
        elif pd.api.types.is_string_dtype(dtype):
            return 'VARCHAR'
        elif pd.api.types.is_datetime64_any_dtype(dtype):
            return 'DATETIME'
        else:
            return 'TEXT'

def parse_directory(directory_path):
    parser = DataParser()
    results = {}
    metadata = {}
    for filename in os.listdir(directory_path):
        filepath = os.path.join(directory_path, filename)
        if os.path.isfile(filepath):
            data, meta = parser.parse_file(filepath)
            results[filename] = data.to_html()  # Ensuring data is always a DataFrame.
            if isinstance(meta, dict):  # Handle single error message
                metadata[filename] = pd.DataFrame([meta]).to_html(index=False)
            else:
                metadata[filename] = pd.DataFrame(meta).to_html(index=False)
    return results, metadata

# Define the directory path and display results.
directory_path = r"C:\Users\robel\Desktop\Different Datasets"
results, metadata = parse_directory(directory_path)
count = 0
max_display = 5
for filename in results:
    if count < max_display:
        print(f"Filename: {filename}")
        print("Data Preview:")
        display(HTML(results[filename]))
        print("Metadata Table:")
        display(HTML(metadata[filename]))
        count += 1


Filename: customers.csv
Data Preview:


,Customer_ID,Name,Email,Phone_Number,Address
0,1,Natalie Davidson,peter05@hotmail.com,001-092-928-4374,"USCGC Larson, FPO AE 01626"
1,2,John Beasley,cory70@mcdonald.com,+1-027-739-9995,"76354 West Greens Apt. 281, Youngfurt, NV 42569"
2,3,Sara Fleming,matthew68@yahoo.com,254.032.1080x29568,"77271 Estrada Mount, South Emma, UT 36419"
3,4,Thomas Moore,sthompson@gmail.com,(605)799-7345x991,"6624 Shepherd Dale Suite 542, East Tony, MS 58880"
4,5,Jacob Snyder,allenbrad@smith.com,701-298-7386x53775,"1532 Welch Well, West Rachelmouth, VA 90134"
5,6,Jill Garrett,bholt@yahoo.com,+1-932-945-4439x7229,"1221 Scott Knoll, Timothymouth, UT 30567"
6,7,Ryan Walters,maryfrank@rowe.com,263.170.8415x012,"6310 Keith Ways, Brendafurt, LA 29519"
7,8,Nicholas Collins,sheripayne@hotmail.com,8019159464,"318 Mark Rapid Suite 846, Port Nicholas, CA 86595"
8,9,Joshua Perez,maria20@gmail.com,(587)534-8847x6245,"755 Caitlyn Fork, West Sergio, AZ 88136"
9,10,Laura Pope,tiffany76@gmail.com,804-674-7389x407,"51139 Destiny Valley, New Diane, NY 32669"


Metadata Table:


Field Name,Data Type,Field Length
Customer_ID,INT,3
Name,VARCHAR,26
Email,VARCHAR,33
Phone_Number,VARCHAR,22
Address,VARCHAR,59


Filename: patient_records.ndjson
Data Preview:


...

Field Name,Data Type,Field Length
Company ID,INT,3
Company Name,VARCHAR,11
Founded Year,INT,4
Products,VARCHAR,25
Categories,VARCHAR,21


Filename: sample_nested_json_data.json
Data Preview:


,id,name,age,contact.email,contact.phones
0,1,Person 1,21,person1@example.com,"[123-456-781, 987-654-321]"
1,2,Person 2,22,person2@example.com,"[123-456-782, 987-654-322]"
2,3,Person 3,23,person3@example.com,"[123-456-783, 987-654-323]"
3,4,Person 4,24,person4@example.com,"[123-456-784, 987-654-324]"
4,5,Person 5,25,person5@example.com,"[123-456-785, 987-654-325]"
5,6,Person 6,26,person6@example.com,"[123-456-786, 987-654-326]"
6,7,Person 7,27,person7@example.com,"[123-456-787, 987-654-327]"
7,8,Person 8,28,person8@example.com,"[123-456-788, 987-654-328]"
8,9,Person 9,29,person9@example.com,"[123-456-789, 987-654-329]"
9,10,Person 10,30,person10@example.com,"[123-456-780, 987-654-320]"


Metadata Table:


Field Name,Data Type,Field Length
id,INT,3
name,VARCHAR,10
age,INT,2
contact.email,VARCHAR,21
contact.phones,VARCHAR,11
